In [ ]:
import feast
import pandas as pd
from datetime import datetime
import yaml

In [ ]:
with open('feature_repo/feature_store.yaml', 'r') as file:
    fs_config_yaml = yaml.safe_load(file)
fs_config = feast.repo_config.RepoConfig(**fs_config_yaml)
fs = feast.FeatureStore(config=fs_config)

In [ ]:
# Let's use the feature service here like we would in production.
# It's better to point to a single feature definition that packages our features.
feature_service = fs.get_feature_service("serving_fs")

In [ ]:
song_properties = pd.read_parquet('../99-data_prep/song_properties.parquet')
favorite_song = song_properties.loc[song_properties["name"]=="Not Like Us"]
favorite_song

In [ ]:
features = fs.get_online_features(
    features=feature_service,
    entity_rows=[
        {
            "spotify_id": favorite_song["spotify_id"].values[0],
        }
    ],
)

In [ ]:
features = features.to_dict()
features

## And we can now send this data to our model

In [ ]:
import requests

In [ ]:
deployed_model_name = "jukebox"
infer_endpoint = "<paste-the-link-here>"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

In [ ]:
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "input",
                "shape": [1, 13],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

In [ ]:
data = [[
    features["is_explicit"][0],
    features["duration_ms"][0],
    features["danceability"][0],
    features["energy"][0],
    features["key"][0],
    features["loudness"][0],
    features["mode"][0],
    features["speechiness"][0],
    features["acousticness"][0],
    features["instrumentalness"][0],
    features["liveness"][0],
    features["valence"][0],
    features["tempo"][0],
]]
prediction = rest_request(data)
prediction